In [9]:
from pathlib import Path
import pandas as pd
from collections import Counter


def save_to_sheet(df, sheet, writer):
    df.to_excel(writer, sheet_name=sheet, index=True, header=True)


main = Path("Xports")
stats_folder_main = Path("stats")

for mode in ("pivot", "LEMMA"):
    stats_folder = stats_folder_main / mode
    jsons = list(main.glob("*/*.json"))
    jsons = [Path("/home/marceau/PycharmProjects/GrewCount/Xports/UD_French-PUD/VERB.json")]

    nb_feuilles = 10

    stats = {}

    for json in jsons:
        corpus = json.parent.name
        query = json.stem

        stats_file = stats_folder / corpus
        stats_file.mkdir(parents=True, exist_ok=True)
        stats_file = stats_file / (query + ".xlsx")

        print(f"{stats_file = }")

        if corpus not in stats:
            stats[corpus] = {}
        stats[corpus][query] = {}

        df = pd.read_json(json)

        if len(df) == 0:
            print(f"{json = }")
            print(f"{df = }")
            continue

        verbes = df[mode]
        verbes_freq = Counter([e.lower() for e in verbes])
        occurences = len(verbes)

        # print(df.head())

        tuplverbes = sorted([(v, verbes_freq[v]) for v in verbes_freq], key=lambda x: x[1], reverse=True)
        tuplverbes = {v: c for v, c in tuplverbes}

        temp = {
            "nb_occurences": occurences,
            "nb_verbes": len(verbes.unique()),
            **tuplverbes,
        }

        feuilles = {}

        for verbe, count in verbes_freq.most_common(nb_feuilles):
            phrases = df[df[mode] == verbe]
            feuilles[verbe] = {}
            feuilles[verbe]["stats"] = {
                "nb_occurences": count,
                "freq": count / occurences,
                "nb_phrases": len(phrases),
            }

            # [(sent_id, left, pivot, right) for sent_id, left, pivot, right in phrases[["sent_id", "left_context", "pivot", "right_context"]].values]
            feuilles[verbe]["phrases"] = phrases.values.tolist()

        with pd.ExcelWriter(stats_file, engine='xlsxwriter') as writer:

            tempdf = pd.DataFrame(temp, index=["", ])
            tempdf = tempdf.T

            save_to_sheet(tempdf, "Globales", writer)

            for verbe, content in feuilles.items():
                tempdf = pd.DataFrame(content["stats"], index=[0])
                save_to_sheet(tempdf, f"{verbe}_stats", writer)

                tempdf = pd.DataFrame(content["phrases"], columns=phrases.columns)
                save_to_sheet(tempdf, f"{verbe}_phrases", writer)


        temp["feuilles"] = feuilles

        stats[corpus][query] = temp

    1/0






stats_file = PosixPath('stats/pivot/UD_French-PUD/VERB.xlsx')


ZeroDivisionError: division by zero

In [10]:
for corpus, content in stats.items():
    stats[corpus]["all"] = {}
    for query, content2 in content.items():
        if query == "all":
            continue
        for k, v in content2.items():
            if k == "feuilles":
                if k not in stats[corpus]["all"]:
                    stats[corpus]["all"][k] = {}
                for verbe, content3 in v.items():
                    if verbe not in stats[corpus]["all"][k]:
                        stats[corpus]["all"][k][verbe] = {}
                        stats[corpus]["all"][k][verbe]["stats"] = {}
                        stats[corpus]["all"][k][verbe]["phrases"] = []

                    for k2, v2 in content3["stats"].items():
                        if k2 not in stats[corpus]["all"][k][verbe]["stats"]:
                            stats[corpus]["all"][k][verbe]["stats"][k2] = 0

                        stats[corpus]["all"][k][verbe]["stats"][k2] += v2

                    stats[corpus]["all"][k][verbe]["phrases"] += content3["phrases"]

            else:
                if k not in stats[corpus]["all"]:
                    stats[corpus]["all"][k] = 0

                stats[corpus]["all"][k] += v

    for corpus in stats:
        stats_file = stats_folder_main / mode / corpus / "corpus.xlsx"
        feuilles = stats[corpus]["all"]["feuilles"]
        with pd.ExcelWriter(stats_file, engine='xlsxwriter') as writer:
            df = pd.DataFrame(stats[corpus]["all"], index=["", ])
            df = df.T
            save_to_sheet(df, "Globales", writer)

            for verbe, content_ in feuilles.items():
                tempdf = pd.DataFrame(content_["stats"], index=[0])
                save_to_sheet(tempdf, f"{verbe}_stats", writer)

                tempdf = pd.DataFrame(content_["phrases"], columns=phrases.columns)
                save_to_sheet(tempdf, f"{verbe}_phrases", writer)

In [ ]:
tempdf

In [ ]:
stats["all"]


In [ ]:
df